# First part of the code: building the churner dataframe.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
df = pd.read_csv('transactions_dataset.csv', sep=';')

In [62]:
df['date_order'] = pd.to_datetime(df['date_order'], format='%Y-%m-%d')
df['date_invoice'] = pd.to_datetime(df['date_invoice'], format='%Y-%m-%d')

In [98]:
# Count the number of unique purchase dates per client
order_counts = df.groupby('client_id')['date_order'].nunique().reset_index()
order_counts.rename(columns={'date_order': 'order_count'}, inplace=True)

# Merge back with df
df = df.merge(order_counts, on='client_id', how='left')

# Keep only clients who ordered more than once
df = df[df['order_count'] > 2]

# Drop the temporary column if not needed
df.drop(columns=['order_count'], inplace=True)

In [99]:
df = df[df['client_id'] != 838349]

In [85]:
print(f"Starting order date: {df['date_order'].min()}")
print(f"Ending order date: {df['date_order'].max()}")

print(f"Starting invoice date: {df['date_invoice'].min()}")
print(f"Ending invoice date: {df['date_invoice'].max()}")

Starting order date: 2017-09-22 00:00:00
Ending order date: 2019-09-22 00:00:00
Starting invoice date: 2016-10-12 00:00:00
Ending invoice date: 2021-07-13 00:00:00


# Dynamic threshold for Churner

In [164]:
# Group by client_id and date_order (to avoid multiple entries per day).

df_daily = df.groupby(['client_id', 'date_order']).agg(
    sales_net=('sales_net', 'sum'),  # Aggregate total spending per day.
    quantity=('quantity', 'sum')  # Aggregate total quantity purchased per day.
).reset_index()

# Compute the time gaps between purchases.

df_daily = df_daily.sort_values(by=['client_id', 'date_order'])  # Sort before diff().
df_daily['time_gap'] = df_daily.groupby('client_id')['date_order'].diff().dt.days

# Compute the mean time gap per client.
  
avg_gap = df_daily.groupby('client_id')['time_gap'].mean().reset_index()
avg_gap.rename(columns={'time_gap': 'avg_time_gap'}, inplace=True)

# Create a customer summary table.

customer_summary = df.groupby('client_id').agg(
    last_purchase=('date_order', 'max'),  # Last recorded invoice date
    total_monetary=('sales_net', 'sum'),  # Total spending
    avg_monetary=('sales_net', 'mean')  # Average spending per order
).reset_index()

# Round the 'total_monetary' columns to the nearest euro.

customer_summary['total_monetary'] = customer_summary['total_monetary'].round(0).astype(int)

# Add a frequency column showing the number of purchases made by each client in total.

frequency = df.groupby('client_id')['date_order'].nunique().reset_index()
frequency.rename(columns={'date_order': 'frequency'}, inplace=True)

# Merge with customer_summary
customer_summary = customer_summary.merge(frequency, on='client_id', how='left')

# Create a recency column: number of days since last purchase.

latest_date = df['date_order'].max()
customer_summary['recency'] = (latest_date - customer_summary['last_purchase']).dt.days

# Merge the customer_summary df with the avg_gap df for time gaps averages.

customer_summary = customer_summary.merge(avg_gap, on='client_id', how='left')

# Set a dynamic churn threshold (x3 the average time gap)

customer_summary['dynamic_threshold'] = customer_summary['avg_time_gap'] * 3 

# Identify churners (recency > dynamic threshold)

customer_summary['churner'] = customer_summary['recency'] > customer_summary['dynamic_threshold']

# Remove the customers having a null or negative total_monetary value.

customer_summary = customer_summary[customer_summary['total_monetary'] > 0]

# Printing results.

number_churners = customer_summary[customer_summary['churner'] == True].shape[0]
number_clients = customer_summary.shape[0]
percentage_churners = round((number_churners / number_clients) * 100)

print(f"There are {number_churners} churners within the {number_clients} clients.")
print(f"{percentage_churners}% clients are churners.")

There are 37761 churners within the 136087 clients.
28% clients are churners.


### Download CSV

In [165]:
customer_summary.to_csv('customer_summary.csv')

In [166]:
df.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051


In [167]:
df['order_channel'].value_counts()

order_channel
at the store                       32001287
by phone                           25382264
online                              5648144
other                                 58384
during the visit of a sales rep       28992
Name: count, dtype: int64